In [55]:
import pandas as pd
import numpy as np
from datetime import datetime

In [56]:
df = pd.read_csv("../../data/trusted/arquivo_anonimizado_v2.csv")
D_CURSO = pd.read_csv('../../data/refined/d_curso.csv')
D_PERIODO = pd.read_csv('../../data/refined/d_periodo.csv')
D_ALUNO = pd.read_csv('../../data/refined/d_aluno.csv')

In [57]:
def obter_ultimo_periodo_cursado(row):
    disciplinas_str = row["disciplinas_cursadas"]
    if pd.isna(disciplinas_str):
        if pd.isna(row['cra_periodo']) and pd.isna(row['cr_periodo']):
            return row['periodo_ingresso_ufrj']
        else:
            periodo = row['cra_periodo'].split(' - ')[0]
            return periodo
    
    disciplinas = disciplinas_str.split("\n")

    periodos = []
    for disciplina in disciplinas:
        periodo = disciplina.split(" - ")[0]
        periodos.append(periodo)

    maior_periodo = max(periodos)
    return maior_periodo.replace("/0", "/1")

# Processar Trancamentos
def processar_trancamentos(row):
    if pd.isna(row):
        return row
    row = row.replace(' ', '')
    row = tuple(row.split('|'))
    return row

def calcular_periodo_integralizado(periodo_ingresso, quant_integralizado):
    ano, semestre = map(int, periodo_ingresso.split('/'))
    quant_integralizado = quant_integralizado-1
    # Se quant_integralizado for ímpar, ajusta semestre e ano
    if quant_integralizado % 2 == 1:
        semestre += 1
        if semestre > 2:
            semestre = 1
            ano += 1
    
    # Calcula o novo ano considerando todos os semestres integralizados
    ano += quant_integralizado // 2

    return f"{ano}/{semestre}"

In [58]:
# Obtém o valor máximo da coluna 'periodoIngressoUFRJ'
periodo_atual = df['periodo_ingresso_ufrj'].max()

# Filtra os dados para obter apenas as linhas com 'situacao_matricula' igual a 'Ativa'
df_situacao_atual = df[(df['situacao_matricula'] == "Ativa") | (df['situacao_matricula'] == "Trancada")].copy()
df_cancel_concluido = df[(df['situacao_matricula'] == "Cancelada") | (df['situacao_matricula'] == "Concluido")].copy()

# Trancamento
df_trancada = df[~df['periodos_trancados'].isna()].copy()
df_trancada['situacao_matricula'] = "Trancada"
df_trancada['periodo'] = df_trancada['periodos_trancados'].apply(processar_trancamentos)

In [59]:
df_situacao_atual['periodo'] = periodo_atual
df_trancada = df_trancada.explode('periodo')
df_cancel_concluido['periodo'] = df.apply(obter_ultimo_periodo_cursado, axis=1)

In [ ]:
colunas_tb_fato = [
    "matricula_dre",
    "periodo",
    "cod_curso_ingresso",
    "curso_ingresso_ufrj",
    "cod_curso_atual",
    "curso_atual",
    "situacao_matricula"
]

In [61]:
df_ativa = df_situacao_atual[colunas_tb_fato]
df_trancada = df_trancada[colunas_tb_fato]
df_cancel_concluido = df_cancel_concluido[colunas_tb_fato]

# Usa a função concat para unir os DataFrames
df_final = pd.concat([df_ativa, df_trancada, df_cancel_concluido], axis=0)

# Reseta o índice se necessário
df_final.reset_index(drop=True, inplace=True)

In [62]:
# Mescla com a tabela D_PERIODO, selecionando apenas a coluna 'sk_d_periodo'
f_situacao_metricula = df_final.merge(
    D_PERIODO[["periodo", "sk_d_periodo"]], on="periodo", how="left"
).drop(columns=["periodo"])

# Mescla com a tabela D_CURSO e remove as colunas desnecessárias
f_situacao_metricula = f_situacao_metricula.merge(
    D_CURSO,
    on=["cod_curso_ingresso", "curso_ingresso_ufrj", "cod_curso_atual", "curso_atual"],
    how="left",
).drop(columns=["cod_curso_ingresso", "curso_ingresso_ufrj", "cod_curso_atual", "curso_atual"])

# Mescla com a tabela D_ALUNO e remove as colunas desnecessárias
f_situacao_metricula = f_situacao_metricula.merge(
    D_ALUNO,
    on=[
        "matricula_dre",
    ],
    how="left",
).drop(
    columns=[
        "nome_completo",
        "matricula_dre",
        "sexo",
        "data_nascimento",
        "periodo_ingresso_curso_atual",
        "periodo_ingresso_ufrj",
        "forma_ingresso",
        "modalidade_cota",
        "nota_enem",
    ],
)

In [64]:
f_situacao_metricula.to_csv('../../data/refined/f_situacao_metricula.csv', index=False)